# Calculate CMIP Indeces

This file 
1. calculates all the indeces for all months for all cesmFullForcingFiles.



In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils.timePeriod as tp



In [2]:
import xarray
import numpy
import matplotlib.pyplot as plt

# 1. Calculate CESM Indeces

import warnings
warnings.filterwarnings('ignore')

For each experiment in historical, calculate indices

In [3]:
experimentSet=['piControl', 'historical', 'ssp126', 'ssp245', 'ssp370','ssp585']


In [4]:
#need to double check about the model run used!
#modelSet=_model.scenarioMIP[:,1]
modelSet=numpy.array([
    #['CSIRO-ARCCSS', 'ACCESS-CM2'],
    #['CSIRO', 'ACCESS-ESM1-5'],
    ['AWI', 'AWI-CM-1-1-MR'],
    #['BCC', 'BCC-CSM2-MR'],
    ['CAMS', 'CAMS-CSM1-0'],
    ['CAS', 'CAS-ESM2-0'],
    ['NCAR', 'CESM2'],
    ['NCAR', 'CESM2-WACCM'],
    ['THU', 'CIESM'],
    ['CMCC', 'CMCC-CM2-SR5'],
    ['CMCC', 'CMCC-ESM2'],
    ['CNRM-CERFACS', 'CNRM-CM6-1'],
    ['CNRM-CERFACS', 'CNRM-CM6-1-HR'],
    ['CNRM-CERFACS', 'CNRM-ESM2-1'],
    ['CCCma', 'CanESM5'],
    ['CCCma', 'CanESM5-CanOE'],
    ['E3SM-Project', 'E3SM-1-1'],
    ['EC-Earth-Consortium', 'EC-Earth3'],
    ['EC-Earth-Consortium', 'EC-Earth3-AerChem'],
    ['EC-Earth-Consortium', 'EC-Earth3-CC'],
    ['EC-Earth-Consortium', 'EC-Earth3-Veg'],
    ['EC-Earth-Consortium', 'EC-Earth3-Veg-LR'],
    ['FIO-QLNM', 'FIO-ESM-2-0'],
    ['NOAA-GFDL', 'GFDL-CM4'],
    ['NOAA-GFDL', 'GFDL-ESM4'],
    ['NASA-GISS', 'GISS-E2-1-G'],
    ['MOHC', 'HadGEM3-GC31-LL'],
    ['MOHC', 'HadGEM3-GC31-MM'],
    ['CCCR-IITM', 'IITM-ESM'],
    ['INM', 'INM-CM4-8'],
    ['INM', 'INM-CM5-0'],
    ['IPSL', 'IPSL-CM5A2-INCA'],
    ['IPSL', 'IPSL-CM6A-LR'],
    ['NIMS-KMA', 'KACE-1-0-G'],
    ['KIOST', 'KIOST-ESM'],
    ['UA', 'MCM-UA-1-0'],
    ['MIROC', 'MIROC-ES2L'],
    ['MIROC', 'MIROC6'],
    ['HAMMOZ-Consortium', 'MPI-ESM-1-2-HAM'],
    ['MPI-M', 'MPI-ESM1-2-HR'],
    ['MPI-M', 'MPI-ESM1-2-LR'],
    ['MRI', 'MRI-ESM2-0'],
    ['NUIST', 'NESM3'],
    ['NCC', 'NorESM2-LM'],
    ['NCC', 'NorESM2-MM'],
    ['MOHC', 'UKESM1-0-LL']
])

In [5]:
for model in modelSet[:,1]: #_model.scenarioMIP[:,1]:
    
    try: 
        print(model)
        #fxDs = fh.loadModelData(model, 'areacella_fx', 'piControl') 
        #SST
        tsDs = fh.loadModelData(model, 'ts_Amon', 'piControl').ts.to_dataset()
        #controlDs=xarray.merge([tsDs.ts, fxDs.areacella])
        controlDs=tsDs.assign_attrs({'project_id':'CMIP'})

        sstClimat=sst.calculateClimatology(controlDs)

        pslControlDs=fh.loadModelData(model, 'psl_Amon', 'piControl')
        pslClimat=psl.calculateClimatology(pslControlDs)

        for experiment in experimentSet: 
            
            try:

                tsDs = fh.loadModelData(model, 'ts_Amon', experiment).ts.to_dataset()
                #sstDs = xarray.merge([tsDs.ts, fxDs.areacella])
                sstDs=tsDs.assign_attrs({'project_id':'CMIP'})

                sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)

                pslDs = fh.loadModelData(model, 'psl_Amon', experiment)
                pslIndex, junk = psl.calculateSamIndex(pslDs, pslClimat)
                indeces = xarray.merge([pslIndex, sstIndex])
                indeces.assign_attrs(climatology='full length of pi Control')
                #print(indeces)
                tp.averageForTimePeriod(indeces).to_netcdf('results/cmipWarmSeasonIndeces/'+model+'_'+experiment + '.nc')

            except Exception as e:
                print(experiment + "not completed")
                print(e)
                
            else:
                print(experiment + ' complete')  
            
    except Exception as e:
        print("Climatology did not calculate")
        print(e)
        

    

AWI-CM-1-1-MR


C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


Ds looks like CMIP6


C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
C:\Anaconda3\lib\site-packages\dask\array\numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


piControl complete
historical complete
ssp126 complete
ssp245 complete


...

ssp370 complete


C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 143 times more chunks
  return self.array[key]
C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 143 times more chunks
  return self.array[key]
C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 143 times more chunks
  return self.array[key]
C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 143 times more chunks
  return self.array[key]
C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To 

ssp585 complete
NESM3
CMIP6/CMIP/NUIST/NESM3/piControl/r1i1p1f1
file not found on AWS
Climatology did not calculate
Files (filter term: psl_Amon_.*?NESM3_piControl_r1i1p1f1_.*?\.nc ) not found, possibly test name is wrong
NorESM2-LM
CMIP6/CMIP/NCC/NorESM2-LM/piControl/r1i1p1f1
file not found on AWS
Climatology did not calculate
Files (filter term: psl_Amon_.*?NorESM2-LM_piControl_r1i1p1f1_.*?\.nc ) not found, possibly test name is wrong
NorESM2-MM


C:\Anaconda3\lib\site-packages\xarray\core\indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


CMIP6/CMIP/NCC/NorESM2-MM/piControl/r1i1p1f1
file not found on AWS
Climatology did not calculate
Files (filter term: psl_Amon_.*?NorESM2-MM_piControl_r1i1p1f1_.*?\.nc ) not found, possibly test name is wrong
UKESM1-0-LL
CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f1
Climatology did not calculate
'NoneType' object is not iterable


This grabs the monthly CESM indeces, and calculates the averages for the time periods of interest (aka the warm season), as set in the _indexDefitionions file

Theres a few examples of output at the bottom to sanity check this is meaningful

The indeces for each month have already been calculated. So loading up the saved versions, and putting them into a single xarray.

In [ ]:
for experiment in experimentSet: 
    print(experiment)
    timePIndeces=list()

    for model in modelSet:
        print(model)
        indecesDs = xarray.open_dataset('results/cmipWarmSeasonIndeces/' + model +'_'+ experiment + '.nc')

        timePIndeces.append(indecesDs)

    results=xarray.concat(timePIndeces, 'model')

    results['model']=modelSet

    results=results.assign_attrs({'experiment':experiment,**indecesDs.attrs,**_index.monthsOfInterest})

    results.to_netcdf('results/cmip6'+experiment+'TradIndeces.nc')

In [ ]:
results

Some sanity checks to see if it looks ok:


In [ ]:
results=xarray.open_dataset('results/cmip5piControlTradIndeces.nc')

In [ ]:
# plotSomething
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

results.nino34.plot(label='nino34')
results.nino34NoDetrend.plot(label='No Detrend')

#Tidy up a bit
plt.title("CMIP Past 1000 Nino34")
plt.ylim(-4,4)
plt.minorticks_on()
plt.legend()

In [ ]:
# A multi-experiment mean?

mean = results.sam.mean(dim='experiment')
std = results.sam.std(dim='experiment')

In [ ]:
plt.figure()
mean.plot()
std.plot()

In [ ]:
# why limit yourself to one index?

mean = results.mean(dim='experiment')
std = results.std(dim='experiment')

In [ ]:
std

# 2. Calculate Compound Years for CESM

In [ ]:
# Here is the data
indecesDs = xarray.open_dataset('results/cmip'+experiment+'TradIndeces.nc')

indecesDs.experiment.values

In [ ]:
indecesDs

# Define a criteria to determine events

So we are going to define events as follows:
- For SST indeces (aka Enso and Iod), this is 1 sigma above/below mean
- For PSL indeces (aka Sam) this is 0.5 sigma above/below mean

For now, using a 850-1850 climatology

In [ ]:
# This should calculate stds for all experiments and all indeces
stdDa = indecesDs.std(dim='year')

# define some criteria that we want to use to identifiy it thats an event
criteriaDa= xarray.merge(
    (stdDa[['nino34','dmi']], stdDa['sam']/2)
)

# Calculate the compounding years and plot an example

In [ ]:
%run utils/compound.ipynb



firePos, fireNeg = applyCriteria(indecesDs, criteriaDa)

firePos=compound(firePos)

fireNeg=compound(fireNeg)

firePos.to_netcdf('results/cmip'+experiment+'FirePos.nc')

fireNeg.to_netcdf('results/cmip'+experiment+'FireNeg.nc')

In [ ]:
firePos

In [ ]:
expToPlot = 'LME.002'

toPlot=3*firePos.all3.sel(experiment=expToPlot)

plt.figure(figsize=(12,12))

plt.vlines(
    x=firePos.year, 
    ymin=0,
    ymax=toPlot,
    label='all3'
)

colors=['orange', 'purple', 'green']
iCol = 0

for iPair in firePos.attrs['pairs']:
    plt.vlines(
        x=firePos.year, 
        ymin=0,
        ymax=2*firePos[iPair].sel(experiment=expToPlot),
        color=colors[iCol],
        label=iPair
    )
    iCol+=1
plt.legend()
plt.xlim(1700,2100)
plt.title(expToPlot+': Pairs and all-three compounds for individal years')